In [1]:
#import bibliotek
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
from csv import writer
import numpy as np
import pandas as pd

In [2]:
#laczenie z chrome i strona
driver = webdriver.Chrome('./chromedriver.exe')
#driver.get("https://europeanleague.football/live-games/210926-hamburg-frankfurt")
driver.get("https://europeanleague.football/live-games/210829-wroclaw-berlin")

<ipython-input-2-43c9f3ec624a>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [3]:
page_info = 'https://europeanleague.football/live-games/210829-wroclaw-berlin'

In [4]:
#przeklikanie cookies
driver.find_element(By.XPATH,'.//*[@id="consentDialog"]/div[2]/div[2]/div/div[2]/div/div[1]/div').click()

In [5]:
#rozklikanie wszystkich list rozwijanych
input_element = driver.find_elements(By.XPATH,'.//*[starts-with(@id, "driveindex")]/div[2]/div[7]')
for item in input_element:
    item.click()
    time.sleep(1)
page_source = driver.page_source

In [6]:
#przejscie do BS4
soup = BeautifulSoup(page_source, 'lxml')

In [7]:
'''for lista in lists:
    play = lista.find('h4').text
    desc = lista.find('div', class_="drives_plays_play_desc").text
    down = lista.find('div', class_="drives_plays_play_next_step").text
    print(play)
    print(desc)
    print(down)'''


'for lista in lists:\n    play = lista.find(\'h4\').text\n    desc = lista.find(\'div\', class_="drives_plays_play_desc").text\n    down = lista.find(\'div\', class_="drives_plays_play_next_step").text\n    print(play)\n    print(desc)\n    print(down)'

In [8]:
lists = soup.find_all(class_="drives_plays_play")

In [9]:
#petla i df
dane = []
columns = ['aa','play','bb','situation','dd','desc','cc']
for lista in lists:
    down = lista.find('div', class_="drives_plays_play_next_step").text
    play = lista.find('h4').text
    desc = lista.find('div', class_="drives_plays_play_desc").text
    dane.append(lista)

df = pd.DataFrame(dane,columns=columns)
df

C:\Users\iza\anaconda3\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,aa,play,bb,situation,dd,desc,cc
0,,[Pass],,[Bryan Zerbe pass incomplete to Seantavius Jon...,,"[[1st], down and , [10], at , [BT19]]",
1,,[Rush],,[Joc Crawford rush for loss of 4 yards to the ...,,"[[2nd], down and , [10], at , [BT19]]",
2,,[Pass],,[Bryan Zerbe pass incomplete to Seantavius Jon...,,"[[3rd], down and , [14], at , [BT15]]",
3,,[Punt],,[Kevin Hummel punt 35 yards to the 50 yardline...,,"[[4th], down and , [14], at , [BT15]]",
4,,[Drive start],,[PANTHERS WROCLAW drive start at 13:57.],,"[[1st], down and , [10], at , [PW50]]",
...,...,...,...,...,...,...,...
217,,[Kickoff],,"[Tadhg Leader kickoff 59 yards to the BT6, Tin...",,"[[1st], down and , [0], at , [PW35]]",
218,,[Drive start],,[BERLIN THUNDER drive start at 02:00.],,"[[1st], down and , [10], at , [BT45]]",
219,,[Pass],,[Calvin Stitt pass incomplete to Emil Drossard...,,"[[1st], down and , [10], at , [BT45]]",
220,,[Pass],,[Calvin Stitt pass incomplete to Tino Ndongo.],,"[[2nd], down and , [10], at , [BT45]]",


In [10]:
#drop zbednych kolumn
df.drop(['aa', 'bb','cc', 'dd'], axis=1,inplace=True)

In [11]:
#rozbicie pozycji na indexy
df[['down','team2','yds_to_go','team4','field_pos']] = pd.DataFrame(df.desc.tolist(), index= df.index)

C:\Users\iza\anaconda3\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [12]:
df = df.explode('play')
df = df.explode('situation')
df = df.explode('down')
df = df.explode('yds_to_go')
df = df.explode('field_pos')

In [13]:
#drop zbednych kolumn
df.drop(['desc', 'team2','team4'], axis=1,inplace=True)

In [46]:
df['home_team'] = page_info.split('-')[2].strip()
df['away_team'] = page_info.split('-')[3].strip()
df['game_date'] = page_info.split('-')[1].strip().split('/')[1].strip()
df['passer'] = np.where(df['play'].str.contains('Pass'), df["situation"].str.split(" ").str[:2].str.join(" "), None)
df['receiver'] = np.where((df['play'].str.contains('Pass')&(df['situation'].str.contains('pass complete'))), 
                           df["situation"].str.split(" ").str[5:7].str.join(" "), None)
df['intended_receiver'] = np.where((df['play'].str.contains('Pass')&(df['situation'].str.contains('pass incomplete'))), 
                           df["situation"].str.split(" ").str[5:7].str.join(" "), None)
df['rusher'] = np.where((df['play'].str.contains('Rush')&(df['situation'].str.contains('rush'))), 
                           df["situation"].str.split(" ").str[0:2].str.join(" "), None)
df['rec_yds'] = np.where((df['play'].str.contains('Pass')&(df['situation'].str.contains('pass complete'))), 
                           df["situation"].str.split(" ").str[8:9].str.join(" "), 0)
df['rush_yds'] = np.where((df['play'].str.contains('Rush')), 
                           df["situation"].str.split(" ").str[4:5].str.join(" "), 0)
df['punt_yds'] = np.where((df['play'].str.contains('Punt')), 
                           df["situation"].str.split(" ").str[3:4].str.join(" "), 0)
df['kickoff_yds'] = np.where((df['play'].str.contains('Kickoff')), 
                           df["situation"].str.split(" ").str[3:4].str.join(" "), 0)
df['fumble'] = np.where((df['situation'].str.contains('fumble')), 1, 0)
df['def_player_action'] = df['situation'].str.extract(r"\((.*?)\)", expand=False)
df['int'] = np.where((df['play'].str.contains('Pass')&(df['situation'].str.contains('intercepted'))), 1, 0)
df['int_player'] = np.where((df['play'].str.contains('Pass')&(df['situation'].str.contains('intercepted'))),
                            df["situation"].str.split(" ").str[5:7].str.join(" "),None)
df['td'] = np.where((df['situation'].str.contains('TOUCHDOWN')), 1, 0)
df['pat'] = np.where((df['play'].str.contains('Point after try')), 1, 0)
df['fg'] = np.where((df['play'].str.contains('Field goal attempt')&(df['situation'].str.contains('GOOD'))), 1, 0)
df['player'] = df["situation"].str.split(" ").str[:2].str.join(" ")
df.player = df.player.apply(lambda x: "Lukas O'Connor" if 'Lokas' in x else x)
df.player = df.player.apply(lambda x: "Jocques Crawford" if 'Joc Crawford' in x else x)
df.player = df.player.apply(lambda x: "Jocques Crawford" if 'Philieas Pasqualini' in x else x)
df['pos_team'] = df.player.map(df_player.set_index('player')['2021_team'].to_dict())

In [30]:
df_player = pd.read_csv('players-file.csv')

In [47]:
df

,play,situation,down,yds_to_go,field_pos,home_team,away_team,game_date,passer,receiver,intended_receiver,rusher,rec_yds,rush_yds,punt_yds,kickoff_yds,fumble,def_player_action,int,int_player,td,pat,fg,player,pos_team
0,Pass,Bryan Zerbe pass incomplete to Seantavius Jone...,1st,10,BT19,wroclaw,berlin,210829,Bryan Zerbe,None,Seantavius Jones,None,0,0,0,0,0,Darius Robinson,0,None,0,0,0,Bryan Zerbe,Berlin Thunder
1,Rush,Joc Crawford rush for loss of 4 yards to the B...,2nd,10,BT19,wroclaw,berlin,210829,None,None,None,Joc Crawford,0,loss,0,0,0,William Lloyd;Adrian Brudny,0,None,0,0,0,Jocques Crawford,Berlin Thunder
2,Pass,Bryan Zerbe pass incomplete to Seantavius Jones.,3rd,14,BT15,wroclaw,berlin,210829,Bryan Zerbe,None,Seantavius Jones.,None,0,0,0,0,0,NaN,0,None,0,0,0,Bryan Zerbe,Berlin Thunder
3,Punt,"Kevin Hummel punt 35 yards to the 50 yardline,...",4th,14,BT15,wroclaw,berlin,210829,None,None,None,None,0,0,35,0,0,NaN,0,None,0,0,0,Kevin Hummel,Berlin Thunder
4,Drive start,PANTHERS WROCLAW drive start at 13:57.,1st,10,PW50,wroclaw,berlin,210829,None,None,None,None,0,0,0,0,0,NaN,0,None,0,0,0,PANTHERS WROCLAW,NaN
5,Pass,Lokas O´Connor pass incomplete to Wiktor Zieba.,1st,10,PW50,wroclaw,berlin,210829,Lokas O´Connor,None,Wiktor Zieba.,None,0,0,0,0,0,NaN,0,None,0,0,0,Lukas O'Connor,Panthers Wroclaw
6,Rush,Philieas Pasqualini rush for 13 yards to the B...,2nd,10,PW50,wroclaw,berlin,210829,None,None,None,Philieas Pasqualini,0,13,0,0,0,Arne Gehrt,0,None,0,0,0,Philieas Pasqualini,NaN
7,Pass,Lokas O´Connor pass incomplete (Kolin Hill).,1st,10,BT37,wroclaw,berlin,210829,Lokas O´Connor,None,Hill).,None,0,0,0,0,0,Kolin Hill,0,None,0,0,0,Lukas O'Connor,Panthers Wroclaw
8,Rush,Philieas Pasqualini rush for 1 yard to the BT3...,2nd,10,BT37,wroclaw,berlin,210829,None,None,None,Philieas Pasqualini,0,1,0,0,0,Tuna Hiller;Wael Nasri,0,None,0,0,0,Philieas Pasqualini,NaN
9,Pass,Lokas O´Connor pass incomplete to Wiktor Zieba.,3rd,9,BT36,wroclaw,berlin,210829,Lokas O´Connor,None,Wiktor Zieba.,None,0,0,0,0,0,NaN,0,None,0,0,0,Lukas O'Connor,Panthers Wroclaw


In [31]:
df_player

,2021_team,2022_team,player,first_name,last_name
0,NaN,Leipzig Kings,AJ Wentland,AJ,Wentland
1,Berlin Thunder,NaN,Aaron Boadu,Aaron,Boadu
2,NaN,NaN,Aaron Ellis,Aaron,Ellis
3,NaN,Berlin Thunder,Aaron Rosemann,Aaron,Rosemann
4,NaN,Barcelona Dragons,Aarón Sola,Aarón,Sola
5,NaN,Istanbul Rams,Abdullah Kilany,Abdullah,Kilany
6,NaN,Rhein Fire,Achim Bah,Achim,Bah
7,NaN,Panthers Wroclaw,Adam Lary,Adam,Lary
8,NaN,Panthers Wroclaw,Adam Skakowski,Adam,Skakowski
9,Berlin Thunder,NaN,Adedayo Odeleye,Adedayo,Odeleye


In [21]:
page_info.split('-')[3].strip()

'berlin'